In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import cython
import tweepy
import json
import requests
import inspect
import re
from tqdm import tqdm

In [12]:
data = pd.read_csv("politicians.csv")
print(data)

usernames = data["Username"].tolist()
ids = data["ids"].tolist()



          Party                      Name  Branch        Username  \
0    Republican             Pete Sessions   House    PeteSessions   
1    Democratic            Debbie Dingell   House   RepDebDingell   
2    Democratic           Pramila Jayapal   House      RepJayapal   
3    Republican               Rob Wittman   House      RobWittman   
4    Democratic           Chellie Pingree   House  chelliepingree   
..          ...                       ...     ...             ...   
521  Republican               Thom Tillis  Senate   SenThomTillis   
522  Republican         Diana Harshbarger   House  RepHarshbarger   
523  Republican          Tommy Tuberville  Senate   SenTuberville   
524  Democratic  Debbie Wasserman Schultz   House    RepDWStweets   
525  Republican              Lisa McClain   House  RepLisaMcClain   

                     ids  
0               24735461  
1             2970279814  
2     815733290955112448  
3               15356407  
4               14984637  
..       

In [13]:
with open("dev_keys_json.json", "r") as f:
    keys = json.load(f)
client = tweepy.Client(bearer_token=keys["Bearer_token"],
                       access_token = keys["API_key"],
                       access_token_secret = keys["API_secret_key"],
                       wait_on_rate_limit=True)



In [ ]:
following_ids = []
following_usernames = []

pbar = tqdm(ids)


i = 0
for id in pbar:
    pbar.set_description(f"{usernames[i]}")
    followings = []

    
    for follow in tweepy.Paginator(client.get_users_following, id = id,
                                  max_results=1000).flatten(limit = 99999):
        followings.append(follow)
    
    these_follow_ids = []
    these_follow_names = []
    
    for f in followings:
        #print(f)
        f_id = f["id"]
        f_username = f["username"]
        
        these_follow_ids.append(f_id)
        these_follow_names.append(f_username)
    #print(these_follow_ids)
    #print(these_follow_names)
    
    following_ids.append(these_follow_ids)
    following_usernames.append(these_follow_names)
    
    i += 1
        
        
print(following_ids)
print(following_usernames)


SenStabenow:  10%|███████████████▎                                                                                                                                     | 54/526 [1:45:20<9:04:07, 69.17s/it]Rate limit exceeded. Sleeping for 893 seconds.


In [ ]:
id_one = data["ids"][0]
following = client.get_users_following(id = id_one)
print(following[0])
print(repr(following[0][0]["id"]))